## 0. 모듈 임포트


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from IPython.display import display

from pathlib import Path
from tqdm import tqdm
import random


## 1. 라벨 데이터 로딩 및 기본 정보 확인


In [ ]:
column_names = [
    "class", "x_center", "y_center", "width", "height",
    "front_key_x", "front_key_y", "front_key_vis",
    "tail_key_x", "tail_key_y", "tail_key_vis",
    "right_key_x", "right_key_y", "right_key_vis",
    "left_key_x", "left_key_y", "left_key_vis"
]


In [ ]:
all_df = {}

for i in range(8, 15):
    path = f'./runs/predict/predict{i}/labels/'
    data = []

    for label_path in Path(path).glob('*.txt'):
        with open(label_path, 'r') as f:
            lines = f.readlines()

        if not lines:
            print(f"⚠️ 파일이 비어 있음: {label_path.name}")
            continue

        for line in lines:
            numbers = list(map(float, line.strip().split()))
            data.append(numbers)

    if not data:
        print(f"⚠️ 폴더 내 라벨 데이터 없음: predict{i}")
        continue

    df = pd.DataFrame(data)

    if df.shape[1] != len(column_names):
        print(f"⚠️ 열 수 불일치: {label_path.name} (columns: {df.shape[1]})")
        continue

    df.columns = column_names
    all_df[f"predict{i}"] = df


In [ ]:
for key in all_df.keys():
    display(all_df[key].info())


In [ ]:
for key in all_df.keys():
    display(all_df[key].describe())


## 2. 방향성 오류 탐지: 키포인트 벡터 기반 필터링


In [ ]:
all_error_df = {}

for pred_key in all_df.keys():
    df = all_df[pred_key]
    df_errors_idx = []
    threshold = 1e-3

    for i, row in df.iterrows():
        vec_forward = np.array([
            row["tail_key_x"] - row["front_key_x"],
            row["tail_key_y"] - row["front_key_y"]
        ])

        vec_rl = np.array([
            row["left_key_x"] - row["right_key_x"],
            row["left_key_y"] - row["right_key_y"]
        ])

        norm_f = np.linalg.norm(vec_forward)
        norm_rl = np.linalg.norm(vec_rl)

        if norm_f == 0 or norm_rl == 0:
            continue

        vec_forward_normalized = vec_forward / norm_f
        vec_rl_normalized = vec_rl / norm_rl
        cross_z = np.cross(vec_forward_normalized, vec_rl_normalized)

        if cross_z >= -threshold:
            df_errors_idx.append(i)

    df_errors = df.loc[df_errors_idx].reset_index(drop=True)
    all_error_df[pred_key] = df_errors


## 3. 방향성 오류 샘플 시각화


In [ ]:
keypoints = ['front', 'tail', 'right', 'left']
colors = {'front': 'red', 'tail': 'blue', 'right': 'green', 'left': 'purple'}
x_scale, y_scale = 3840, 2160

num_cols = 4
num_rows = 3
num_samples = num_cols * num_rows

for pred_key in all_error_df.keys():
    df_errors = all_error_df[pred_key]

    if len(df_errors) <= num_samples:
        selected_samples = df_errors
    else:
        selected_samples = df_errors.sample(num_samples).reset_index(drop=True)

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 5, num_rows * 5))
    axes = axes.flatten()

    for idx, row in enumerate(selected_samples.itertuples()):
        ax = axes[idx]
        for kpt in keypoints:
            ax.scatter(
                getattr(row, f"{kpt}_key_x") * x_scale,
                getattr(row, f"{kpt}_key_y") * y_scale,
                color=colors[kpt],
                label=kpt if idx == 0 else "",
                s=120
            )

        ax.plot(
            [row.front_key_x * x_scale, row.tail_key_x * x_scale],
            [row.front_key_y * y_scale, row.tail_key_y * y_scale],
            'k--', alpha=0.6
        )
        ax.plot(
            [row.right_key_x * x_scale, row.left_key_x * x_scale],
            [row.right_key_y * y_scale, row.left_key_y * y_scale],
            'k--', alpha=0.6
        )

        ax.set_title(f"{pred_key} Sample {row.Index}")
        ax.set_xlim(0, x_scale)
        ax.set_ylim(0, y_scale)
        ax.grid(True)
        ax.set_xticks([])
        ax.set_yticks([])

    handles, labels = axes[0].get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    fig.legend(by_label.values(), by_label.keys(), loc='upper right')

    for idx in range(len(selected_samples), len(axes)):
        fig.delaxes(axes[idx])

    plt.tight_layout()
    plt.show()


In [ ]:
correction_rows = []

for frame_name, df in all_df.items():
    for idx, row in df.iterrows():
        vec_forward = np.array([
            row["tail_key_x"] - row["front_key_x"],
            row["tail_key_y"] - row["front_key_y"]
        ])
        vec_rl = np.array([
            row["left_key_x"] - row["right_key_x"],
            row["left_key_y"] - row["right_key_y"]
        ])

        norm_f = np.linalg.norm(vec_forward)
        norm_rl = np.linalg.norm(vec_rl)

        if norm_f == 0 or norm_rl == 0:
            continue

        vec_forward_normalized = vec_forward / norm_f
        vec_rl_normalized = vec_rl / norm_rl

        cross_z = np.cross(vec_forward_normalized, vec_rl_normalized)

        if cross_z >= -threshold:
            row_data = row.to_dict()
            row_data["file_name"] = frame_name
            correction_rows.append(row_data)

df_corrections = pd.DataFrame(correction_rows).reset_index(drop=True)


In [ ]:
num_cols = 4
num_rows = 3
num_samples = num_cols * num_rows

if len(df_corrections) <= num_samples:
    selected_samples = df_corrections
else:
    selected_samples = df_corrections.sample(num_samples).reset_index(drop=True)

fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 5, num_rows * 5))
axes = axes.flatten()

for idx, row in enumerate(selected_samples.itertuples()):
    ax = axes[idx]
    for key in keypoints:
        ax.scatter(
            getattr(row, f"{key}_key_x") * x_scale,
            getattr(row, f"{key}_key_y") * y_scale,
            color=colors[key],
            label=key if idx == 0 else "",
            s=120
        )

    ax.plot(
        [row.front_key_x * x_scale, row.tail_key_x * x_scale],
        [row.front_key_y * y_scale, row.tail_key_y * y_scale],
        'k--', alpha=0.6
    )
    ax.plot(
        [row.right_key_x * x_scale, row.left_key_x * x_scale],
        [row.right_key_y * y_scale, row.left_key_y * y_scale],
        'k--', alpha=0.6
    )

    ax.set_title(f"{row.file_name}")
    ax.set_xlim(0, x_scale)
    ax.set_ylim(0, y_scale)
    ax.grid(True)
    ax.set_xticks([])
    ax.set_yticks([])

handles, labels = axes[0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc='upper right')

for idx in range(len(selected_samples), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()


In [ ]:
correction_rows = []

for frame_name, df in all_df.items():
    for idx, row in df.iterrows():
        vec_forward = np.array([
            row["tail_key_x"] - row["front_key_x"],
            row["tail_key_y"] - row["front_key_y"]
        ])
        vec_rl = np.array([
            row["left_key_x"] - row["right_key_x"],
            row["left_key_y"] - row["right_key_y"]
        ])

        norm_f = np.linalg.norm(vec_forward)
        norm_rl = np.linalg.norm(vec_rl)

        if norm_f == 0 or norm_rl == 0:
            continue

        vec_forward_normalized = vec_forward / norm_f
        vec_rl_normalized = vec_rl / norm_rl

        cross_z = np.cross(vec_forward_normalized, vec_rl_normalized)

        if cross_z >= -threshold:
            row_data = row.to_dict()
            row_data["file_name"] = frame_name
            row_data["original_index"] = idx
            correction_rows.append(row_data)

original_df = pd.DataFrame(correction_rows).reset_index(drop=True)


In [ ]:
corrected_rows = []
for row_data in correction_rows:
    corrected_data = row_data.copy()
    corrected_data["right_key_x"], corrected_data["left_key_x"] = corrected_data["left_key_x"], corrected_data["right_key_x"]
    corrected_data["right_key_y"], corrected_data["left_key_y"] = corrected_data["left_key_y"], corrected_data["right_key_y"]
    corrected_rows.append(corrected_data)

corrected_df = pd.DataFrame(corrected_rows).reset_index(drop=True)

unique_files = original_df['file_name'].unique()


In [ ]:
for file_name in unique_files:
    file_original = original_df[original_df['file_name'] == file_name]

    if len(file_original) > 3:
        file_original = file_original.sample(3, random_state=42).reset_index(drop=True)
    else:
        file_original = file_original.reset_index(drop=True)

    indices = file_original["original_index"].tolist()
    file_corrected = corrected_df[(corrected_df['file_name'] == file_name) &
                                  (corrected_df['original_index'].isin(indices))].reset_index(drop=True)

    num_samples = len(file_original)
    fig, axes = plt.subplots(2, num_samples, figsize=(num_samples * 5, 10))

    if num_samples == 1:
        axes = np.array([[axes[0]], [axes[1]]])

    for idx, row in enumerate(file_original.itertuples()):
        ax = axes[0, idx]
        for key in keypoints:
            ax.scatter(
                getattr(row, f"{key}_key_x") * x_scale,
                getattr(row, f"{key}_key_y") * y_scale,
                color=colors[key],
                label=key if idx == 0 else "",
                s=120
            )
        ax.plot(
            [row.front_key_x * x_scale, row.tail_key_x * x_scale],
            [row.front_key_y * y_scale, row.tail_key_y * y_scale],
            'k--', alpha=0.6
        )
        ax.plot(
            [row.right_key_x * x_scale, row.left_key_x * x_scale],
            [row.right_key_y * y_scale, row.left_key_y * y_scale],
            'k--', alpha=0.6
        )
        ax.set_title(f"Original Line {row.original_index + 1}")
        ax.set_xlim(0, x_scale)
        ax.set_ylim(0, y_scale)
        ax.grid(True)
        ax.set_xticks([])
        ax.set_yticks([])

    for idx, row in enumerate(file_corrected.itertuples()):
        ax = axes[1, idx]
        for key in keypoints:
            ax.scatter(
                getattr(row, f"{key}_key_x") * x_scale,
                getattr(row, f"{key}_key_y") * y_scale,
                color=colors[key],
                s=120
            )
        ax.plot(
            [row.front_key_x * x_scale, row.tail_key_x * x_scale],
            [row.front_key_y * y_scale, row.tail_key_y * y_scale],
            'k--', alpha=0.6
        )
        ax.plot(
            [row.right_key_x * x_scale, row.left_key_x * x_scale],
            [row.right_key_y * y_scale, row.left_key_y * y_scale],
            'k--', alpha=0.6
        )
        ax.set_title(f"Corrected Line {row.original_index + 1}")
        ax.set_xlim(0, x_scale)
        ax.set_ylim(0, y_scale)
        ax.grid(True)
        ax.set_xticks([])
        ax.set_yticks([])

    fig.suptitle(f"File: {file_name}", fontsize=16)

    handles, labels = axes[0, 0].get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    fig.legend(by_label.values(), by_label.keys(), loc='upper right')

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()
